# Social Computing - Summer 2018
# Exercise 3: Collaborative Filtering Recommender System

Write a simple collaborative filtering movie recommender in python. The input (attached to this exercise) is a subset of the MovieLens dataset (movielens.org), which contains 1862 movies, and 100K ratings. Find the dataset Piazza. The input is provided in two data files: u.item: is a listing of movies. Each row represents a movie with its attributes separated by ‘|’. We are only interested in the first two attributes which are the movie ID and the movie name. The second data file is u.data, which contains the movie ratings by users. A single row represents one user’s rating for one movie, and the attributes are (from left to right): user ID, movie ID, rating, and timestamp. 

<b>Exercise</b><br>
The entry point to your recommendation engine should be a python method called “recommend” that takes a user ID, and paths to the two aforementioned data files. The method should return the top twenty recommended movies for that user <u><b>(The movies should not have been rated by that user before)</b></u>. The output should be a list of python tuples (sorted by recommended movies' expected ratings: highest first). Each tuple has the following two attributes: movie name, expected rating. You are free to design your recommendation engine the way you want but straightforward collborative filtering is highly recommended. Make sure that the code is clean, readable, and well documented. Test the code for user 15!

### The Collaborative Filtering Recommender
The idea of collaborative filtering is to find similar users to the target user. The items highly rated by those users are likely to be favorited by the target user. Therefore, the main problem is to find the list of users similar to the target user. There are several ways of measuring similarity. We could e.g. use cosine similarity but one of the simplest measures is Euclidean distance. Your program should calculate the absolute Euclidean distance between the target user and all other users in the dataset and calculate the expected rating for the target user for each movie in the dataset based on the forumla:

$$r_{ui} = \frac{\sum_{v \in N_i(u)} w_{uv}r_{vi}}{\sum_{v \in N_i(u)} w_{uv}}$$

Where $r_{ui}$ is the expected recommendation of item i for target user u. $N_i(u)$ is the set of similar users to target user u for the designated item i. $w_{uv}$ is the similarity score between users u and v (used as a weighting factor) for $r_{vi}$ which is the rating of user v for item i. 
The curious student should refer to: Ricci et al (eds.) "Recommender Systems Handbook", Springer 2011, for more details

In [8]:
# Method has following parameters:
# path_u_item: path to u.item as a string
# HINT: movie_dictionary = {movie_id: movie_name}
# TODO Parse u.item


import pandas as pd
import numpy as np
def create_movie_dictionary(path_to_item):
    m_cols = ['movie_id', 'title']
    df_items = pd.read_csv('u.item', sep='|',names=m_cols, usecols=range(2), encoding='latin-1')
    movie_dictionary = {}
    movie_dictionary = dict(zip(df_items.movie_id, df_items.title))
    return movie_dictionary

# Create a dictionary: {user_id: {movie_id: user_rating}}
def create_user_rating_dictionary(path_input):
    user_rating_dictionary = {}
    for line in open(path_input):
        row = line.strip().split('\t')
        user_id, movie_id, rating, timestamp = int(row[0]), int(row[1]), int(row[2]), int(row[3])
        try:
            user_rating_dictionary.setdefault(user_id, {})
            user_rating_dictionary[user_id][movie_id] = rating
        except KeyError:
            print( "key error found! " + user_id + " " + movie_id)
    return user_rating_dictionary


# Using Euclidean distance to calculate similarity score
def calculate_similarity_score(ratings, user_id1, user_id2):
    # TODO Find common movies rated by both users

    ratings1 = ratings.get(int(user_id1))  ## Finding the movies rated by user_id1
    ratings2 = ratings.get(int(user_id2))  ## Finding the movies rated by user_id2
    both_ratings = [ratings1, ratings2]

    ## Calculating the common movies rated by both users ##
    common_movies = []
    common_movies = set(ratings1.keys())
    for d in both_ratings[1:]:
        common_movies.intersection_update(set(d.keys()))
    common_movies = list(common_movies)

    if len(common_movies) == 0:  # no common ratings between two users. Similarity is 0
        return 0

    # TODO Calculate Euclidean distance between two users based on their common ratings

    sum_of_squares_of_differences = 0
    for movie_id in common_movies:
    # TODO Accumulate the sum of squares of differences in ratings between the two users for the same movie
        rating_u1 = ratings1.get(movie_id)  ## Getting the corresponding rating for user1 for the common movie
        rating_u2 = ratings2.get(movie_id)  ## Getting the corresponding rating for user2 for the common movie
        squares_of_differences = (rating_u1-rating_u2) * (rating_u1-rating_u2)
        sum_of_squares_of_differences += squares_of_differences
        

    return 1 / (1 + sum_of_squares_of_differences)

   
    

def cf_recommend(ratings, target_user_id):
    weighted_ratings = {}  # {movie_id: weighted_rating}
    similarity_scores = {}  # {movie_id: similarity_score}
    recommended_movie_list = []  # Each element is a tuple (estimated_rating, movie_id)
    for user_id in ratings:
        if user_id != target_user_id:
            similarity_score = calculate_similarity_score(ratings, target_user_id, user_id)
            if similarity_score > 0:
                for movie_id in ratings[user_id]:
                    # Movie was not recommended by the target user before
                    if movie_id not in ratings[int(target_user_id)]:
                        rating_by_user = ratings[user_id].get(movie_id)
                        weighted_rating = rating_by_user * similarity_score
                        weighted_ratings[movie_id] = weighted_rating
                        similarity_scores[movie_id] = similarity_score
                # TODO Accumulate the weighted rating for that movie
                # The weighted rating of the movie = user_id's rating of that movie * similarity
                # score between that user and the target_user
                # of that user to the target_user
                # TODO Accumulate the similarity scores of all users who rated that movie
                        
    for movie_id in weighted_ratings: # TODO for each movie
        estimated_rating = weighted_ratings[movie_id]/similarity_scores[movie_id] # TODO Weighted_rating/sigma (similarity scores of users who rated that movie)
#         print("estimated rating of movie_id {} is {}".format(movie_id, estimated_rating))
        recommended_movie_list.append((estimated_rating, movie_id))

    # TODO Sort the list
#     return sorted(recommended_movie_list, reverse= True) # List of recommended movies for user_id from highest to lowest estimated rating
    recommended_movie_list.sort(key=lambda tup: tup[0], reverse = True)
    return recommended_movie_list

# Testing implementation
movie_dict = create_movie_dictionary("u.item")
ratings = create_user_rating_dictionary("u.data")
recommended_movies = cf_recommend(ratings, '15')
top_twenty = []
for estimated_rating, movie_id in recommended_movies[:20]:
    top_twenty.append((movie_dict[movie_id], estimated_rating))
print (np.array(top_twenty))


[['Enchanted April (1991)' '5.000000000000001']
 ['My Left Foot (1989)' '5.000000000000001']
 ['Piano, The (1993)' '5.000000000000001']
 ['Umbrellas of Cherbourg, The (Parapluies de Cherbourg, Les) (1964)'
  '5.000000000000001']
 ['The Deadly Cure (1996)' '5.000000000000001']
 ['Nico Icon (1995)' '5.000000000000001']
 ["Muriel's Wedding (1994)" '5.0']
 ['Being There (1979)' '5.0']
 ['Cold Comfort Farm (1995)' '5.0']
 ['Beautiful Girls (1996)' '5.0']
 ['Princess Bride, The (1987)' '5.0']
 ['Fifth Element, The (1997)' '5.0']
 ['Beauty and the Beast (1991)' '5.0']
 ['Devil in a Blue Dress (1995)' '5.0']
 ['Set It Off (1996)' '5.0']
 ['Lion King, The (1994)' '5.0']
 ['Rock, The (1996)' '5.0']
 ['Hunchback of Notre Dame, The (1996)' '5.0']
 ['Aladdin (1992)' '5.0']
 ['Fugitive, The (1993)' '5.0']]
